In [1]:
import numpy as np
from typing import Tuple
import scipy.constants as co
import h5py
from tqdm import tqdm

# import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
# pio.templates.default = "plotly_dark"

## Startup
Be aware of some lazy coding. DO not copy this stupidly into the main program.

The next code blocks are just a preparation to get interparticle distances AND to get forces array. Later we will use this in detail.

In [2]:
def load_data(filename:str) -> Tuple[np.ndarray, np.ndarray, float, np.ndarray, np.ndarray, float]:
    """Loads the datafile indicated and returns basic properties

    Args:
        filename (str): path to the .hdf5 file

    Returns:
        Tuple:
            pos (np.ndarray): full system position array all timesteps
            force (np.ndarray): full system force array all timesteps
            L (float): boxsize
            N (np.ndarray): Atom amounts per type
            t (np.ndarray): time array
    """
    df = h5py.File(filename)
    skips =  1 # or: df['/input/incar/ML_OUTBLOCK'][()]
    L = df['results/positions/lattice_vectors'][()][0, 0]
    pos = df['intermediate/ion_dynamics/position_ions'][()]*L
    force = df['intermediate/ion_dynamics/forces'][()]
    N = df['results/positions/number_ion_types'][()]
    T = df['intermediate/ion_dynamics/energies'][()][:, 3].mean()
    dt = df['/input/incar/POTIM'][()]*skips
    t =np.arange(pos.shape[0])*dt
    df.close()
    return pos, force, L, N, t, T

def rdf_force_shapshot(r, d, F, L, bins):
    storage_array=np.zeros(np.size(bins), dtype=np.float64)

    F_dot_r = np.sum(F*r, axis=1)/d  # F dot rxyz/r (strength of F in the direction of r)
    dp = F_dot_r/d**2  # (F dot r_vec)/r_rad^3
    dp[(r[:, 0]>L/2)+(r[:, 1]>L/2)+(r[:, 2]>L/2)]=0  # filter
    
    # the next part is from revelsmd package, adjusted for my code
    digtized_array = np.digitize(d, bins)-1
    dp[digtized_array==np.size(bins)-1] = 0
    
    storage_array[(np.size(bins)-1)]= np.sum(dp[(digtized_array==np.size(bins)-1)]) #conduct heaviside for our first bin
    for l in range(np.size(bins)-2,-1,-1):
        storage_array[l]= np.sum(dp[(digtized_array==l)])#conduct subsequent heavisides with a rolling sum
    return storage_array

def run_rdf(pos, force, L, N, t, T,delr=.01,start=0,stop=-1,period=1, rmax=True, from_zero=True):
    """
    This is the master function for running a force RDF.

    args:
    temp (float): Temperature of the system
    delr (float): The spacing between radial points in an RDF (this is not a bin width as this is not a histogram but a heaviside)
    kwargs:
    start (int): The first frame for which the radial distribution function will be calculated
    stop (int): The last value for which the radial distribution function will be calculated
    period (int): The jumps made between sampled frames
    rmax (float): The maximum radial position defaults to follow the minimum image convention
    from_zero (bHOl): A boolian value if True the Heviside is taken from zero if false it is take from rmax

    returns:
    A 2 dimensional numpy array of r values and acommpanying rdf values

    """
    N_H = N[0]
    N_O = N[1]
    N_K = N[2]
    
    H_i = np.arange(N_H)
    O_i = np.arange(N_H, N_H + N_O)
    K_i = np.arange(N_H + N_O, N_H + N_O + N_K)

    pos_H = pos[:, H_i, :]
    pos_O = pos[:, O_i, :]
    pos_K = pos[:, K_i, :]

    force_H = force[:, H_i, :]
    force_O = force[:, O_i, :]
    force_K = force[:, K_i, :]
    
    
    idx_KK = np.triu_indices(N_K, k=1)
    idx_KO = np.mgrid[0:N_K, 0:N_O].reshape(2, N_K*N_O)
    idx_HO = np.mgrid[0:N_H, 0:N_O].reshape(2, N_H*N_O)
    idx_OO = np.triu_indices(N_O, k=1)

    if start > pos.shape[0]:
        print('First frame index exceeds frames in trajectory')
        return
    if stop > pos.shape[0]:
        print('Final frame index exceeds frames in trajectory')
        return

    # find how many steps to run
    to_run=range(int(start%pos.shape[0]),int(stop%pos.shape[0]),period)

    # set the bins
    if rmax is True:
        bins= np.arange(0,L/2,delr)
    else:
        bins= np.arange(0,float(rmax),delr)
    
    rdf_HO = np.zeros(np.size(bins), dtype=np.float64)
    rdf_OO = np.zeros_like(rdf_HO)
    rdf_KK = np.zeros_like(rdf_HO)
    for frame_count in tqdm(to_run):
        # Hydrogen-Oxygen
        r_HO = (pos_O[frame_count, idx_HO[1], :] - pos_H[frame_count, idx_HO[0], :] +L/2)%L - L/2
        d_HO = np.sqrt(np.sum(r_HO**2, axis=1))
        F_HO = force_O[frame_count, idx_HO[1], :] - force_H[frame_count, idx_HO[0], :]  # Fj - Fi
        rdf_HO += rdf_force_shapshot(r_HO, d_HO, F_HO, L, bins)
        
        # Oxygen-Oxygen
        r_OO = (pos_O[frame_count, idx_OO[1], :] - pos_O[frame_count, idx_OO[0], :] + L/2)%L - L/2  # distance vec pbc&mic OO
        d_OO = np.sqrt(np.sum(r_OO**2, axis=1))  # distance radially pbc&mic
        F_OO = force_O[frame_count, idx_OO[1], :] - force_O[frame_count, idx_OO[0], :]  # Fj - Fi
        rdf_OO+=rdf_force_shapshot(r_OO, d_OO, F_OO, L, bins)
        
        # Potasium-Potasium
        r_KK = (pos_K[frame_count, idx_KK[1], :] - pos_K[frame_count, idx_KK[0], :] + L/2)%L - L/2  # distance vec pbc&mic OO
        d_KK = np.sqrt(np.sum(r_KK**2, axis=1))  # distance radially pbc&mic
        F_KK = force_K[frame_count, idx_KK[1], :] - force_K[frame_count, idx_KK[0], :]  # Fj - Fi
        rdf_KK+=rdf_force_shapshot(r_KK, d_KK, F_KK, L, bins)

    rescale_geo = 8*np.pi*len(to_run)*(co.k/co.eV)*T
    
    # Hydrogen-Oxygen
    prefactor_HO = L**3/(idx_HO[0].shape[0])
    rdf_HO *= prefactor_HO/rescale_geo
    # Oxygen-Oxygen
    prefactor_OO = L**3/(idx_OO[0].shape[0])
    rdf_OO *= prefactor_OO/rescale_geo
    # Potasium-Potasium
    prefactor_KK = L**3/(idx_KK[0].shape[0])
    rdf_KK *= prefactor_KK/rescale_geo
    if from_zero == True:
        rdf_HO = np.cumsum(rdf_HO)
        rdf_OO = np.cumsum(rdf_OO)
        rdf_KK = np.cumsum(rdf_KK)
    else:
        rdf_HO = 1-np.cumsum(rdf_HO[::-1])[::-1]
        rdf_OO = 1-np.cumsum(rdf_OO[::-1])[::-1]
        rdf_KK = 1-np.cumsum(rdf_KK[::-1])[::-1]
    return np.array([bins, rdf_HO, rdf_OO,  rdf_KK])

def run_rdf_better(pos, force, L, N, t, T,delr=.01,start=0,stop=-1,period=1, rmax=True, from_zero=True):
    """
    This is the master function for running a force RDF.

    args:
    temp (float): Temperature of the system
    delr (float): The spacing between radial points in an RDF (this is not a bin width as this is not a histogram but a heaviside)
    kwargs:
    start (int): The first frame for which the radial distribution function will be calculated
    stop (int): The last value for which the radial distribution function will be calculated
    period (int): The jumps made between sampled frames
    rmax (float): The maximum radial position defaults to follow the minimum image convention
    from_zero (bool): A boolian value if True the Heviside is taken from zero if false it is take from rmax

    returns:
    A 2 dimensional numpy array of r values and acommpanying rdf values

    """
    N_H = N[0]
    N_O = N[1]
    N_K = N[2]
    
    H_i = np.arange(N_H)
    O_i = np.arange(N_H, N_H + N_O)
    K_i = np.arange(N_H + N_O, N_H + N_O + N_K)

    pos_H = pos[:, H_i, :]
    pos_O = pos[:, O_i, :]
    pos_K = pos[:, K_i, :]

    force_H = force[:, H_i, :]
    force_O = force[:, O_i, :]
    force_K = force[:, K_i, :]
    
    idx_HO = np.mgrid[0:N_H, 0:N_O].reshape(2, N_H*N_O)
    idx_OO = np.triu_indices(N_O, k=1)
    idx_KK = np.triu_indices(N_K, k=1)
    idx_KO = np.mgrid[0:N_K, 0:N_O].reshape(2, N_K*N_O)

    if start > pos.shape[0]:
        print('First frame index exceeds frames in trajectory')
        return
    if stop > pos.shape[0]:
        print('Final frame index exceeds frames in trajectory')
        return

    # find how many steps to run
    to_run=range(int(start%pos.shape[0]),int(stop%pos.shape[0]),period)

    # set the bins
    if rmax is True:
        bins= np.arange(0,L/2,delr)
    else:
        bins= np.arange(0,float(rmax),delr)
    
    store_HO = np.zeros((len(to_run), np.size(bins)))
    rdf_HO = np.zeros(np.size(bins), dtype=np.float64)
    store_OO = np.zeros_like(store_HO)
    rdf_OO = np.zeros_like(rdf_HO)
    store_KK = np.zeros_like(store_HO)
    rdf_KK = np.zeros_like(rdf_HO)
    for (i, frame_count) in enumerate(tqdm(to_run)):
        # Hydrogen-Oxygen
        r_HO = (pos_O[frame_count, idx_HO[1], :] - pos_H[frame_count, idx_HO[0], :] +L/2)%L - L/2
        d_HO = np.sqrt(np.sum(r_HO**2, axis=1))
        F_HO = force_O[frame_count, idx_HO[1], :] - force_H[frame_count, idx_HO[0], :]  # Fj - Fi
        this_frame = rdf_force_shapshot(r_HO, d_HO, F_HO, L, bins)
        rdf_HO += this_frame
        store_HO[i] = this_frame

        # Oxygen-Oxygen
        r_OO = (pos_O[frame_count, idx_OO[1], :] - pos_O[frame_count, idx_OO[0], :] + L/2)%L - L/2  # distance vec pbc&mic OO
        d_OO = np.sqrt(np.sum(r_OO**2, axis=1))  # distance radially pbc&mic
        F_OO = force_O[frame_count, idx_OO[1], :] - force_O[frame_count, idx_OO[0], :]  # Fj - Fi
        this_frame = rdf_force_shapshot(r_OO, d_OO, F_OO, L, bins)
        rdf_OO += this_frame
        store_OO[i] = this_frame
        
        # Potasium-Potasium
        r_KK = (pos_K[frame_count, idx_KK[1], :] - pos_K[frame_count, idx_KK[0], :] + L/2)%L - L/2  # distance vec pbc&mic OO
        d_KK = np.sqrt(np.sum(r_KK**2, axis=1))  # distance radially pbc&mic
        F_KK = force_K[frame_count, idx_KK[1], :] - force_K[frame_count, idx_KK[0], :]  # Fj - Fi
        this_frame = rdf_force_shapshot(r_KK, d_KK, F_KK, L, bins)
        rdf_KK += this_frame
        store_KK[i] = this_frame
    
    # NEED TO SUM MPI CORES HERE
    # Hydrogen-Hydrogen
    rdf_HO = rdf_rescale(store_HO, rdf_HO, idx_HO, L, T)
    rdf_OO = rdf_rescale(store_OO, rdf_OO, idx_OO, L, T)
    rdf_KK = rdf_rescale(store_KK, rdf_KK, idx_KK, L, T)
    bins =  (bins[1:] + bins[:-1])/2
    return bins, rdf_HO, rdf_OO, rdf_KK

def rdf_rescale(store, rdf, idx, L, T):
    rescale_geo =8*np.pi*(co.k/co.eV)*T
    prefactor = L**3/(idx[0].shape[0])
    store = np.array(store)*prefactor/rescale_geo
    store_zero = np.array(np.cumsum(store, axis=1))[:,:-1]
    store_inf = np.array(1-np.cumsum(store[:,::-1], axis=1)[:,::-1][:,1:])
    store_delta = store_inf - store_zero
    
    # total parts
    rescale_geo *= len(store)
    rdf *= prefactor/rescale_geo
    rdf_zero = np.array(np.cumsum(rdf)[:-1])
    rdf_inf = np.array(1-np.cumsum(rdf[::-1])[::-1][1:])
    rdf_delta = rdf_inf - rdf_zero

    # varience part
    var_del=np.mean((store_delta-rdf_delta)**2,axis=0)
    cov_inf=np.mean((store_delta-rdf_delta)*(store_inf-rdf_inf),axis=0)
    weights = cov_inf/var_del
    rdf = np.mean(store_inf*(1-weights)+(store_zero*weights), axis=0)
    return np.array([rdf, rdf_zero, rdf_inf])
    
def rdf_traditional_shapshot(r,d, F, L, bins):
    rdf_state = np.histogram(d, bins=bins)[0]
    return rdf_state

def run_rdf_traditional(pos, force, L, N, t, T,delr=.01,start=0,stop=-1,period=1, rmax=True):
    N_H = N[0]
    N_O = N[1]
    N_K = N[2]
    
    H_i = np.arange(N_H)
    O_i = np.arange(N_H, N_H + N_O)
    K_i = np.arange(N_H + N_O, N_H + N_O + N_K)

    pos_H = pos[:, H_i, :]
    pos_O = pos[:, O_i, :]
    pos_K = pos[:, K_i, :]

    force_H = force[:, H_i, :]
    force_O = force[:, O_i, :]
    force_K = force[:, K_i, :]
    
    
    idx_KK = np.triu_indices(N_K, k=1)
    idx_KO = np.mgrid[0:N_K, 0:N_O].reshape(2, N_K*N_O)
    idx_HO = np.mgrid[0:N_H, 0:N_O].reshape(2, N_H*N_O)
    idx_OO = np.triu_indices(N_O, k=1)

    if start > pos.shape[0]:
        print('First frame index exceeds frames in trajectory')
        return
    if stop > pos.shape[0]:
        print('Final frame index exceeds frames in trajectory')
        return

    # find how many steps to run
    to_run=range(int(start%pos.shape[0]),int(stop%pos.shape[0]),period)

    # set the bins
    if rmax is True:
        bins= np.arange(0,L/2,delr)
    else:
        bins= np.arange(0,float(rmax),delr)
    
    rdf_HO = np.zeros(np.size(bins)-1, dtype=np.float64)
    rdf_OO = np.zeros_like(rdf_HO)
    rdf_KK = np.zeros_like(rdf_HO)
    for frame_count in tqdm(to_run):
        # Hydrogen-Oxygen
        r_HO = (pos_O[frame_count, idx_HO[1], :] - pos_H[frame_count, idx_HO[0], :] +L/2)%L - L/2
        d_HO = np.sqrt(np.sum(r_HO**2, axis=1))
        rdf_HO += rdf_traditional_shapshot(0, d_HO, 0, 0, bins)  # zeros are dummies here
        
        # Oxygen-Oxygen
        r_OO = (pos_O[frame_count, idx_OO[1], :] - pos_O[frame_count, idx_OO[0], :] + L/2)%L - L/2  # distance vec pbc&mic OO
        d_OO = np.sqrt(np.sum(r_OO*r_OO, axis=1))  # distance radially pbc&mic
        rdf_OO += rdf_traditional_shapshot(0, d_OO, 0, 0, bins)  # zeros are dummies here
        
        # Potasium-Potasium
        r_KK = (pos_K[frame_count, idx_KK[1], :] - pos_K[frame_count, idx_KK[0], :] + L/2)%L - L/2  # distance vec pbc&mic OO
        d_KK = np.sqrt(np.sum(r_KK*r_KK, axis=1))  # distance radially pbc&mic
        rdf_KK += rdf_traditional_shapshot(0, d_KK, 0, 0, bins)  # zeros are dummies here

    # recompute bins to centerpoints
    bins = (bins[1:] + bins[:-1])/2
    prefactor_geometry = L**3/(4*np.pi*(bins**2)*(bins[1]-bins[0]))
    
    # Hydrogen-Oxygen
    prefactor_HO = 1/(len(to_run)*idx_HO[1].shape[0])
    rdf_HO *= prefactor_HO*prefactor_geometry

    # Oxygen-Oxygen
    prefactor_OO = 1/(len(to_run)*idx_OO[1].shape[0])
    rdf_OO *= prefactor_OO*prefactor_geometry
    
    # Potasium-Potasium
    prefactor_KK = 1/(len(to_run)*idx_KK[1].shape[0])
    rdf_KK *= prefactor_KK*prefactor_geometry
    return np.array([bins, rdf_HO, rdf_OO, rdf_KK])

In [3]:
file = r"test_output/combined_simulation/vaspout4.h5"
pos, force, L, N, t, T = load_data(filename=file)

## Set properties

In [4]:
i = 10  # investigate timestep 10

pos_i = pos[i, :, :]
force_i = force[i, :, :]
O_i  = np.arange(N[0], N[0] + N[1])
bins = np.linspace(0, L/2, 250)

# Now we can test force rdf code
for the first step, we do this without any function and code it purely. Lets do oxygen-oxygen for the learning first, look into this code https://github.com/user200000/revelsmd/blob/main/revelsMD/revels_rdf.py

In [5]:
skips = 25
rdf0 = run_rdf_traditional(pos, force, L, N, t, T, delr=0.2, period=skips)
rdf1 = run_rdf_traditional(pos, force, L, N, t, T, period=skips)
bins, rdf_HO, rdf_OO, rdf_KK = run_rdf_better(pos, force, L, N, t, T, period=skips)

100%|██████████| 200/200 [00:09<00:00, 22.16it/s]


In [6]:
f = go.FigureWidget()
f.add_scatter(x=rdf0[0, :], y=rdf0[1, :], mode='markers', marker=dict(size=10), name='traditional rough bins')
f.add_scatter(x=rdf1[0, :], y=rdf1[1, :], mode='markers', marker=dict(size=4), name='traditional same bins')

f.add_scatter(x=bins, y=rdf_HO[0, :], name='combined, reduced covariance')
f.add_scatter(x=bins, y=rdf_HO[1, :], name='from zero')
f.add_scatter(x=bins, y=rdf_HO[2, :], name='from infinity (L/2)')
f.update_layout(
    title='force rdf Hydrogen-Oxygen',
    xaxis=dict(title='radial distance r / [Angstrom]',
               range=[0.8, 7.6]),
    yaxis=dict(title='occurance g(r)',
               range=[0, 2.2])  # or 34
)
f.show()

In [7]:
f = go.FigureWidget()
f.add_scatter(x=rdf0[0, :], y=rdf0[2, :], mode='markers', marker=dict(size=10), name='traditional rough bins')
f.add_scatter(x=rdf1[0, :], y=rdf1[2, :], mode='markers', marker=dict(size=4), name='traditional same bins')

f.add_scatter(x=bins, y=rdf_OO[0, :], name='combined, reduced covariance')
f.add_scatter(x=bins, y=rdf_OO[1, :], name='from zero')
f.add_scatter(x=bins, y=rdf_OO[2, :], name='from infinity (L/2)')

f.update_layout(
    title='force rdf Oxygen-Oxygen',
    xaxis=dict(title='radial distance r / [Angstrom]',
               range=[0.8, 7.6]),
    yaxis=dict(title='occurance g(r)',
               range=[0, 2.2])
)
f.show()

In [8]:
f = go.FigureWidget()
f.add_scatter(x=rdf0[0, :], y=rdf0[3, :], mode='markers', marker=dict(size=10), name='traditional rough bins')
f.add_scatter(x=rdf1[0, :], y=rdf1[3, :], mode='markers', marker=dict(size=4), name='traditional same bins')

f.add_scatter(x=bins, y=rdf_KK[0, :], name='combined, reduced covariance')
f.add_scatter(x=bins, y=rdf_KK[1, :], name='from zero')
f.add_scatter(x=bins, y=rdf_KK[2, :], name='from infinity (L/2)')

f.update_layout(
    title='force rdf Potasium-Potasium',
    xaxis=dict(title='radial distance r / [Angstrom]',
               range=[0.8, 7.6]),
    yaxis=dict(title='occurance g(r)',
               range=[0, 3])
)
f.show()